In [1]:
#Normal PyTorch model with 4-bit quantization

import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, bias=True)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(6, 16, 5, bias=True)
        self.relu2 = nn.ReLU()
        self.fc1 = nn.Linear(16 * 5 * 5, 120, bias=True)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84, bias=True)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10, bias=True)

    def forward(self, x):
        out = self.relu1(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = self.relu2(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.relu3(self.fc1(out))
        out = self.relu4(self.fc2(out))
        out = self.fc3(out)
        return out

model = LeNet()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Target device: " + str(device))

# model.to(device); # The semicolon is for not printing the model.


In [6]:
import numpy as np
import torchvision
import torchvision.transforms as transforms
import os

def quantize_image(image):
    """Quantize and binarize an image."""
    image = image.astype(np.float32)
    image = np.floor(image / (256/2))  # Example: reducing to 4-bit quantization
    return image.astype(np.uint8)

def save_dataset_as_npz(data, labels, filename):
    """Save the dataset as a .npz file."""
    np.savez_compressed(filename, data=data, labels=labels)

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Quantize the images
train_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in train_dataset])
train_labels = np.array(train_dataset.targets)

test_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in test_dataset])
test_labels = np.array(test_dataset.targets)

# Save the datasets
os.makedirs('./quantized_data', exist_ok=True)
save_dataset_as_npz(train_images, train_labels, './quantized_data/cifar10_train.npz')
save_dataset_as_npz(test_images, test_labels, './quantized_data/cifar10_test.npz')

Files already downloaded and verified
Files already downloaded and verified


In [12]:
data = np.load('./quantized_data/cifar10_train.npz')
# print(data.files)
print(data['data'].shape)
print(data['labels'].shape)
print(data['data'][328][15])
print(data['labels'][328])

(50000, 32, 32, 3)
(50000,)
[[ 6  7  9]
 [ 5  7  9]
 [ 6  7  8]
 [ 9  9 10]
 [13 12 12]
 [13 10  9]
 [11  9  8]
 [11 10 10]
 [13 13 13]
 [12 13 13]
 [12 12 12]
 [14 14 14]
 [14 14 13]
 [10 10 11]
 [ 7  8  8]
 [ 8  9  9]
 [ 8  9  9]
 [ 8  8  9]
 [ 9  9  9]
 [10 10 10]
 [13 13 13]
 [12 12 13]
 [10 11 11]
 [13 13 12]
 [11 11 10]
 [ 8  8  9]
 [ 7  8  9]
 [ 6  8  9]
 [ 6  7  9]
 [ 5  7  8]
 [ 5  7  8]
 [ 5  7  8]]
8


In [15]:
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class CIFAR10QuantizedDataset(Dataset):
    def __init__(self, npz_file):
        data = np.load(npz_file)
        self.images = data['data']
        self.labels = data['labels']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0
        label = self.labels[idx]
        image = torch.tensor(image.transpose(2, 0, 1))  # HWC to CHW format
        label = torch.tensor(label, dtype=torch.long)
        return image, label

# Load the quantized dataset
train_dataset = CIFAR10QuantizedDataset('./quantized_data/cifar10_train.npz')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CIFAR10QuantizedDataset('./quantized_data/cifar10_test.npz')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:    # print every 100 mini-batches
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

    print('Finished Training')

train(model, train_loader, criterion, optimizer)


Epoch 1, Batch 100, Loss: 1.1644
Epoch 1, Batch 200, Loss: 1.1405
Epoch 1, Batch 300, Loss: 1.1578
Epoch 1, Batch 400, Loss: 1.1530
Epoch 1, Batch 500, Loss: 1.1531


KeyboardInterrupt: 

In [11]:
def test(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on the test dataset: {accuracy:.2f}%')

test(model, test_loader)

Accuracy on the test dataset: 51.18%
